# Homework 6
任课老师：胡俊峰

负责助教：苏亚鲁、李浩然

注意：仅需要提交.ipynb文件，请**不要**将下发压缩包中的其他文件一并交上。

截止日期：3月18日24点

## 第零部分 代码理解
### 0.1 阅读代码预测结果，分析结果是否如预期，并按要求填充代码行，实现功能。

In [1]:
import pandas as pd
df = pd.DataFrame([[1, 2], [3, 4]], columns = ['a','b'])
print(df)
print(df['a']) 

   a  b
0  1  2
1  3  4
0    1
1    3
Name: a, dtype: int64


In [2]:
# 理解那个是0号索引
dft = df.T           
print(dft)
print(dft[1]['a'])  #  print(dft['a'][1] ?)

   0  1
a  1  3
b  2  4
3


In [11]:
df2 = pd.DataFrame([[5, 6], [7, 8]], columns = ['a','b'])
df3 = pd.concat([df,df2])  # 尝试用一行代码，df1，df2的横向拼接？
df3_=df.join(df2,rsuffix='_1')
print(df3)
print(df3_)

   a  b
0  1  2
1  3  4
0  5  6
1  7  8
   a  b  a_1  b_1
0  1  2    5    6
1  3  4    7    8


In [12]:
# 请填充一行代码，实现打印 ‘b’列的值 <6 的df3的子视图
df3.loc[df3.b<6,['a','b']]

,a,b
0,1,2
1,3,4


In [13]:
df = pd.DataFrame([[4, 9]] * 3, columns=['A', 'B'])
print(df)

   A  B
0  4  9
1  4  9
2  4  9


dataframe.apply(function,axis)对一行或一列做出一些操作（axis=1遍历行，axis=0遍历列），可以使用匿名函数。

In [14]:
import numpy as np
print(df.apply(np.sum))
print(df.apply(np.sum, axis=1))

A    12
B    27
dtype: int64
0    13
1    13
2    13
dtype: int64


In [15]:
df.apply(lambda x: [1, 2], axis=1)

0    [1, 2]
1    [1, 2]
2    [1, 2]
dtype: object

In [16]:
df.apply(lambda x: [1, 2], axis=1, result_type='expand')

,0,1
0,1,2
1,1,2
2,1,2


### 0.2 请认真阅读并运行以下示例代码，并思考C/S的多进程、多线程以及下面代码中select实现方式的优缺点。
注：文件夹下的example_client.py为本思考题对应的客户端文件。client.py为第一部分网络编程对应的客户端文件。（两个客户端均无需改动）

In [ ]:
import select
from socket import *
import sys
import queue

# 生成socket对象
server = socket(AF_INET, SOCK_STREAM)
server.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
server.setblocking(False)

# 绑定地址，设置监听
server.bind(('localhost',9999))
server.listen(5)

# 将自己也放进待监测列表里
inputs = [server, ]
outputs = []
message_queues = {}

while True:
    rlist, wlist, elist = select.select(inputs,outputs,inputs) #如果没有任何fd就绪,那程序就会一直阻塞在这里

    for r in rlist:  # 遍历已经可以准备读取数据的 fd
        if r is server: # 如果这个 fd 是server，即 server 有数据待接收读取，说明有新的客户端连接过来了
            conn, client_addr = r.accept()
            print("new connection from",client_addr)
            conn.setblocking(False)
            inputs.append(conn) # 将这个新的客户端连接添加到检测的列表中
            message_queues[conn] = queue.Queue() # 用队列存储客户端发送来的数据，等待服务器统一返回数据

        else:          # 这个可读的 r 不是服务器，那就是某个客户端。就是说客户端发送数据过来了，这些数据处于待读取状态
            try:       # 异常处理，这是为了防止客户端异常断开报错（比如手动关掉客户端黑窗口，服务器也会跟着报错退出）
                data = r.recv(1024)
                if data:    # 根据判断data是否为空，判断客户端是否断开
                    print("收到来自[%s]的数据:" % r.getpeername()[0], data)
                    message_queues[r].put(data)   # 收到的数据先放到queue里,一会返回给客户端
                    if r not in outputs:
                        outputs.append(r)     # 放进可写的fd列表中，表明这些 fd 已经准备好去发送数据了。
                else:   # 如果数据为空，表明客户端断开了
                    print('客户端断开了')
                    if r in outputs:
                        outputs.remove(r)    #  清理已断开的连接
                    inputs.remove(r)         # 清理已断开的连接
                    del message_queues[r]    # 清理已断开的连接
            except ConnectionResetError:     # 如果报错，说明客户端断开了
                print("客户端异常断开了", r)
                if r in outputs:
                    outputs.remove(r)   # 清理已断开的连接
                inputs.remove(r)        # 清理已断开的连接
                del message_queues[r]  # 清理已断开的连接

    for w in wlist:       # 遍历可写的 fd 列表，即准备好发送数据的那些fd
        # 判断队列是否为空
        try :
            next_msg = message_queues[w].get_nowait()
        except queue.Empty:
            # print("client [%s]" % w.getpeername()[0], "queue is empty..")
            outputs.remove(w)
        # 队列不为空，就把队列中的数据改成大写，原样发回去
        else:
            # print("sending msg to [%s]"% w.getpeername()[0], next_msg)
            w.send(next_msg.upper())

    for e in elist:   # 处理报错的 fd
        e.close()
        print("Error occured in ",e.getpeername())
        inputs.remove(e)
        if e in outputs:
            outputs.remove(e)
        del message_queues[e]

## 第一部分：网络编程 (2.5分)

### 请将下列代码补充完整，从而实现如下功能：
**功能：**
* 为每个socket注册用户名，并检查用户名是否已经存在。若已经存在，则发送'Username already exists!'；否则发送'Welcome!'。(1.5分)
* 将注册成功的用户发送的信息，转发给其他的注册成功的用户。(1分)

**注：**
当补充完整代码后，可以运行client.py进行检验，其预期的输出为：\
connecting to localhost port 服务器端口 \
Welcome!to 第一个客户端端口 \
Username already exists!to 第二个客户端端口 \
Welcome!to 第二个客户端端口 \
Hello everybody!to 第二个客户端端口 

In [2]:
from socket import *
from select import select
import queue

class Server:
    def __init__(self, host, port):
        self.server = socket(AF_INET, SOCK_STREAM)
        self.server.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
        self.server.setblocking(False)

        self.server.bind((host, port))
        self.server.listen(5)
        self.rlist = [self.server]
        self.wlist = []
        self.message_queues = {}
        self.users = {}  # 用于判断socket是否已经注册用户名，self.users[user]=False or name_str

    def accept(self):
        while True:
            rs, ws, xs = select(self.rlist, self.wlist, self.rlist)
            # 第一个参数是我们需要监听可读的套接字, 第二个参数是我们需要监听可写的套接字, 第三个参数使我们需要监听异常的套接字
            # 返回可读，可写，异常的套接字列表
            for r in rs:
                if r is self.server:
                    # 可读的若是服务器套接字，则说明有新用户请求连接，否则是老用户在发消息
                    # 服务器接受客户端连接
                    conn, addr = self.server.accept()
                    print("new connection from",addr)
                    conn.setblocking(False)
                    # 将客户端对象也加入到监听的列表中, 当客户端发送消息时 select 将触发
                    self.rlist.append(conn)
                    # 为连接的客户端单独创建一个消息队列，用来保存客户端发送的消息
                    self.message_queues[conn] = queue.Queue()
                    self.users[conn] = False
                    
                else:
                    try:
                        # 服务器接受客户端的消息并转发给所有客户端
                        data = r.recv(1024)
                        if data:
                            # 将当前线程加入可写线程备选中
                            if r not in self.wlist:
                                self.wlist.append(r)
                            print("收到来自[%s]的数据:" % r.getpeername()[1], data.decode('utf-8'))
                            if self.users[r]!=False:  # 将已注册用户的信息广播给其他已注册用户
                                # TODO
                                '''
                                for user in self.users:
                                    print(user.getpeername()[1])
                                    print(self.users[user])
                                print(" ")
                                '''
                                
                                for user in self.users:
                                    #print(self.message_queues[user].empty())
                                    if self.users[user] and self.users[user]!=self.users[r]:
                                        #print(user.getpeername()[1])
                                        
                                        # 这一步格外重要，需要激活其他用户的可写性
                                        if user not in self.wlist:
                                            self.wlist.append(user)
                                        # 将数据放到列表中，待会一次性处理
                                        self.message_queues[user].put_nowait(data)
                                    #print(self.message_queues[user].empty())
                                        
                            else:  # 对未注册用户发送的昵称进行检验，并进行注册
                                # TODO
                                flag=0
                                for user in self.users:
                                    if self.users[user]==data.decode('utf-8'):
                                        flag=1
                                        break
                                if flag==1:
                                    self.message_queues[r].put_nowait('Username already exists!'.encode('utf-8'))
                                else:
                                    self.message_queues[r].put_nowait('Welcome!'.encode('utf-8'))
                                    self.users[r]=data.decode('utf-8')
                            
                                
                                
                        else:
                            print('客户端断开了')
                            if r in self.wlist:
                                self.wlist.remove(r)   # 清理已断开的连接
                            self.rlist.remove(r)        # 清理已断开的连接
                            del self.message_queues[r]  # 清理已断开的连接
                    except ConnectionResetError:
                        print("客户端异常断开了", r)
                        if r in self.wlist:
                            self.wlist.remove(r)   # 清理已断开的连接
                        self.rlist.remove(r)        # 清理已断开的连接
                        del self.message_queues[r]  # 清理已断开的连接
            '''
            for l in self.wlist:
                print(l.getpeername()[1])
            print("")
            '''
            
            for w in ws:       # 遍历可写的 fd 列表，即准备好发送数据的那些fd
                # 判断队列是否为空
                #print(w.getpeername()[1])
                try :
                    next_msg = self.message_queues[w].get_nowait()
                except queue.Empty:
                    #print("client [%s]" % w.getpeername()[0], "queue is empty..")
                    self.wlist.remove(w)
                # 队列不为空，就把队列中的数据改成大写，原样发回去
                else:
                    print("发送数据到 [%s]"% str(w.getpeername()[0])+str(w.getpeername()[1]), next_msg)
                    w.send(next_msg)

            for e in xs:   # 处理报错的 fd
                e.close()
                print("Error occured in ",e.getpeername())
                self.rlist.remove(e)
                if e in self.wlist:
                    self.wlist.remove(e)
                del self.message_queues[e]
            
            '''
            for user in self.users:
                print(user.getpeername()[1])
                print(self.users[user])
            print(" ")
            '''
            
            
            

if __name__=='__main__':
    host = '0.0.0.0'      # 主机名
    port = 9999           # 端口号
    server = Server(host,port)
    server.accept()

OSError: [Errno 48] Address already in use

## 第二部分 Pandas基础练习（4.5分）

### 2.1 基本操作

In [17]:
import pandas as pd
import numpy as np

data = {
    'age': [24, 53, 23, 24, 33, 42, 57, 57, 36, 29, 53, 39],
    'gender': ['M', 'F', 'M', 'M', 'F', 'M', 'M', 'M', 'M', 'M', 'M', 'F'],
    'occupation': ['technician', 'other', 'writer', 'technician', 'technician', 'executive', 'administrator', 'administrator', 'administrator', 'student', 'lawyer', 'other'],
    'zip_code': ['85711', '94043', '32067', '43537', '15213', '98101', '91344', '91344', '05201', '01002', '90703', '30329']
}

创建一组从1开始递增的id，作为行索引，由data生成一个DataFrame。并以gender分组，打印每组的前两条。（0.5分）

In [18]:
# TODO
index=[x for x in range(1,1+len(data['age']))]
df=pd.DataFrame(data=data,index=index)
gender_group=df.groupby('gender')
print(gender_group.get_group('M')[0:2])
print('')
print(gender_group.get_group('F')[0:2])


   age gender  occupation zip_code
1   24      M  technician    85711
3   23      M      writer    32067

   age gender  occupation zip_code
2   53      F       other    94043
5   33      F  technician    15213


删除DataFrame中重复的行（保留一行），再删除所有occupation为"other"的行。（0.5分）

In [19]:
# TODO
df=df[df['occupation']!='other'].drop_duplicates(keep='first')
df

,age,gender,occupation,zip_code
1,24,M,technician,85711
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,technician,15213
6,42,M,executive,98101
7,57,M,administrator,91344
9,36,M,administrator,05201
10,29,M,student,01002
11,53,M,lawyer,90703


以两种方式访问DataFrame的第一行。（0.5分）

In [20]:
# TODO
print(df.loc[1,:])
print(df.iloc[0,:])

age                   24
gender                 M
occupation    technician
zip_code           85711
Name: 1, dtype: object
age                   24
gender                 M
occupation    technician
zip_code           85711
Name: 1, dtype: object


以两种方式访问DataFrame的"occupation"列。（0.5分）

In [21]:
# TODO
print(df['occupation'])
print(df.occupation)

1        technician
3            writer
4        technician
5        technician
6         executive
7     administrator
9     administrator
10          student
11           lawyer
Name: occupation, dtype: object
1        technician
3            writer
4        technician
5        technician
6         executive
7     administrator
9     administrator
10          student
11           lawyer
Name: occupation, dtype: object


打印出DataFrame中出现次数最多的职业名称（0.5分）

In [22]:
# TODO
print(df.groupby('occupation')['occupation'].count().idxmax())

technician


将data_2转换成DataFrame，与上一个DataFrame按照"zip_code"这一列合并，按两种合并方式：（0.5分）

1.合并后结果保留两个DataFrame的并集；

2.合并后只保留data_2的所有数据。

In [23]:
data_2 = {
    'zip_code': ['85711', '94043', '15213', '98101', '30329'],
    'name': ['Alan', 'Alice', 'Abigail', 'Bob', 'Dave']
}
df2 = pd.DataFrame(data_2)

# TODO
# 方式一
print(pd.merge(df, df2, on='zip_code', how='outer'))
# 方式二
print(pd.merge(df,df2,on='zip_code',how='right'))

     age gender     occupation zip_code     name
0   24.0      M     technician    85711     Alan
1   23.0      M         writer    32067      NaN
2   24.0      M     technician    43537      NaN
3   33.0      F     technician    15213  Abigail
4   42.0      M      executive    98101      Bob
5   57.0      M  administrator    91344      NaN
6   36.0      M  administrator    05201      NaN
7   29.0      M        student    01002      NaN
8   53.0      M         lawyer    90703      NaN
9    NaN    NaN            NaN    94043    Alice
10   NaN    NaN            NaN    30329     Dave
    age gender  occupation zip_code     name
0  24.0      M  technician    85711     Alan
1   NaN    NaN         NaN    94043    Alice
2  33.0      F  technician    15213  Abigail
3  42.0      M   executive    98101      Bob
4   NaN    NaN         NaN    30329     Dave


### 2.2 多级索引

In [24]:
# 构建多级索引DataFrame
# int数据由 np.random.randint(0, 150, size=(6, 3) 产生
# from tuples, from_arrays, from_product
import numpy as np
import pandas as pd

df3 = pd.DataFrame(np.random.randint(0, 150, size=(6, 3)),
                  columns=["Python", "C++", "Java"],
                  index=pd.MultiIndex.from_product([['张三', '李四', '王五'],
                                                  ['期中', '期末']]))
df3

Python  C++  Java
张三 期中      28  108    19
   期末      75   84    53
李四 期中       1   82    80
   期末     105  148    53
王五 期中     122   54    54
   期末       9    7    75

实现变换df3为三重索引的serial。(0.5分)

In [25]:
# TODO
s=pd.Series(df3.stack())
print(s)

张三  期中  Python     28
        C++       108
        Java       19
    期末  Python     75
        C++        84
        Java       53
李四  期中  Python      1
        C++        82
        Java       80
    期末  Python    105
        C++       148
        Java       53
王五  期中  Python    122
        C++        54
        Java       54
    期末  Python      9
        C++         7
        Java       75
dtype: int64


按期中40%，期末60%的比例，计算输出每位同学每门功课的总评成绩表。（0.5分）

In [27]:
# TODO
data={}
index=df3.index
for i in range(0,int(len(index)/2)):
    data[(index[2*i][0])]=0.4*df3.loc[index[2*i]]+0.6*df3.loc[index[2*i+1]]
grades=pd.DataFrame(data)
grades




,张三,李四,王五
Python,56.2,63.4,54.2
C++,93.6,121.6,25.8
Java,39.4,63.8,66.6


计算输出每门功课的最高分，最低分，以及平均分数表（课程名做行索引，列名用最高分，最低分，平均分数）。（0.5分）

In [28]:
# TODO
all_data={}
ave_score=grades.T.mean()
max_score=grades.T.max()
min_score=grades.T.min()
all_df=pd.DataFrame({'最高分':max_score,'最低分':min_score,'平均分数':ave_score})
all_df


,最高分,最低分,平均分数
Python,63.4,54.2,57.933333
C++,121.6,25.8,80.333333
Java,66.6,39.4,56.600000


## 第三部分 Pandas数据分析（3分）

使用Pandas的read_csv函数读取 IBM_2006-01-01_to_2018-01-01.csv 中的数据，得到一个DataFrame，通过shape和info()查看数据基本信息，并打印该数据的前10行观察一下。该数据列索引对应的分别是：日期，开盘价，最高价，最低价，收盘价，成交量，股票名称。（0.5分）

In [29]:
import pandas as pd

IBM_file = 'IBM_2006-01-01_to_2018-01-01.csv'
# TODO
IBM_info=pd.read_csv(IBM_file)
print(f"shape:{IBM_info.shape}\n")
print(IBM_info.info())
IBM_info.head(10)


shape:(3020, 7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3020 entries, 0 to 3019
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    3020 non-null   object 
 1   Open    3019 non-null   float64
 2   High    3020 non-null   float64
 3   Low     3019 non-null   float64
 4   Close   3020 non-null   float64
 5   Volume  3020 non-null   int64  
 6   Name    3020 non-null   object 
dtypes: float64(4), int64(1), object(2)
memory usage: 165.3+ KB
None


,Date,Open,High,Low,Close,Volume,Name
0,2006-01-03,82.45,82.55,80.81,82.06,11715200,IBM
1,2006-01-04,82.20,82.50,81.33,81.95,9840600,IBM
2,2006-01-05,81.40,82.90,81.00,82.50,7213500,IBM
3,2006-01-06,83.95,85.03,83.41,84.95,8197400,IBM
4,2006-01-09,84.10,84.25,83.38,83.73,6858200,IBM
5,2006-01-10,83.15,84.12,83.12,84.07,5701000,IBM
6,2006-01-11,84.37,84.81,83.40,84.17,5776500,IBM
7,2006-01-12,83.82,83.96,83.40,83.57,4926500,IBM
8,2006-01-13,83.00,83.45,82.50,83.17,6921700,IBM
9,2006-01-17,82.80,83.16,82.54,83.00,8761700,IBM


观察到Open与Low这两列相比其他列都缺了一项，请删除DataFrame中所有不完整的数据。（0.5分）

In [30]:
# TODO
IBM_info.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
print(IBM_info.info())
IBM_info

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3019 entries, 0 to 3019
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    3019 non-null   object 
 1   Open    3019 non-null   float64
 2   High    3019 non-null   float64
 3   Low     3019 non-null   float64
 4   Close   3019 non-null   float64
 5   Volume  3019 non-null   int64  
 6   Name    3019 non-null   object 
dtypes: float64(4), int64(1), object(2)
memory usage: 188.7+ KB
None


,Date,Open,High,Low,Close,Volume,Name
0,2006-01-03,82.45,82.55,80.81,82.06,11715200,IBM
1,2006-01-04,82.20,82.50,81.33,81.95,9840600,IBM
2,2006-01-05,81.40,82.90,81.00,82.50,7213500,IBM
3,2006-01-06,83.95,85.03,83.41,84.95,8197400,IBM
4,2006-01-09,84.10,84.25,83.38,83.73,6858200,IBM
...,...,...,...,...,...,...,...
3015,2017-12-22,151.82,153.00,151.50,152.50,2990583,IBM
3016,2017-12-26,152.51,153.86,152.50,152.83,2479017,IBM
3017,2017-12-27,152.95,153.18,152.61,153.13,2149257,IBM
3018,2017-12-28,153.20,154.12,153.20,154.04,2687624,IBM


由于股市周末和节假日不开盘，日期为2006-01-06的下一条数据就是2006-01-09，缺少了中间两天的数据。请将所有缺少的数据按照以下规则补充：成交量（volume）填充为零，开盘价等价格与前一天（最近的有数据的一天）保持相同。（0.5分）

提示：使用pandas.date_range生成时间序列

In [31]:
# TODO
from numpy import NaN


dates=pd.date_range('20060103','20171229')
dates
cnt=len(IBM_info)+1
#print(dates[0],IBM_info['Date'][0])
#print(str(dates[0])[0:10]==IBM_info['Date'][0])
#print(IBM_info)
for date in dates:
    if str(date)[0:10] not in list(IBM_info['Date']):
        #print(date)
        IBM_info.loc[cnt]=np.NaN
        IBM_info.loc[cnt,'Date']=str(date)[0:10]
        cnt+=1
#print(IBM_info)
IBM_info=IBM_info.sort_values(by='Date').reset_index().drop('index',axis=1)
#print(IBM_info)
IBM_info['Volume']=IBM_info['Volume'].fillna(0)
#print(IBM_info)
IBM_info=IBM_info.fillna(method='ffill')
#print(IBM_info)
IBM_info


,Date,Open,High,Low,Close,Volume,Name
0,2006-01-03,82.45,82.55,80.81,82.06,11715200.0,IBM
1,2006-01-04,82.20,82.50,81.33,81.95,9840600.0,IBM
2,2006-01-05,81.40,82.90,81.00,82.50,7213500.0,IBM
3,2006-01-06,83.95,85.03,83.41,84.95,8197400.0,IBM
4,2006-01-07,83.95,85.03,83.41,84.95,0.0,IBM
...,...,...,...,...,...,...,...
4374,2017-12-25,151.82,153.00,151.50,152.50,0.0,IBM
4375,2017-12-26,152.51,153.86,152.50,152.83,2479017.0,IBM
4376,2017-12-27,152.95,153.18,152.61,153.13,2149257.0,IBM
4377,2017-12-28,153.20,154.12,153.20,154.04,2687624.0,IBM


在DataFrame中按月份聚合，计算一年当中每个月的成交量总和，以及每个月收盘价的次高值。(1分)

提示：可以通过apply方法实现自定义的聚合函数。（有多个相同值的话，这里先简单地按照排完序取下标第二个就行了）

In [32]:
# TODO
IBM_info['Month']=IBM_info['Date'].apply(lambda x:x[0:7])
month_group=IBM_info.\
    groupby(by=['Month']).\
        apply(lambda x:x.sort_values('Close',ascending=True))

volume_sum=month_group['Volume'].\
    groupby(level=0).sum().\
        reset_index().\
            rename(columns={'Volume':'成交量总和'})
second_high_close=month_group['Close'].\
    groupby(level=0).apply(lambda x:x[1]).\
        reset_index().\
            rename(columns={'Close':'收盘次高价'})
print(volume_sum)
print(second_high_close)

       Month        成交量总和
0    2006-01  146611300.0
1    2006-02  111061200.0
2    2006-03  111811000.0
3    2006-04  114030800.0
4    2006-05  113238100.0
..       ...          ...
139  2017-08   80295410.0
140  2017-09   78281970.0
141  2017-10  121897687.0
142  2017-11   89776130.0
143  2017-12   88788538.0

[144 rows x 2 columns]
       Month   收盘次高价
0    2006-01   80.85
1    2006-02   79.85
2    2006-03   79.94
3    2006-04   81.16
4    2006-05   79.83
..       ...     ...
139  2017-08  139.70
140  2017-09  142.45
141  2017-10  146.48
142  2017-11  148.40
143  2017-12  152.50

[144 rows x 2 columns]


读取AMZN_2006-01-01_to_2018-01-01.csv，其中包含了Amazon的股票数据。
请以Date作为索引，'IBM'和'AMZN'作为列索引，构建一个Dataframe，其中的数据为两家公司每天的股票收盘价。（0.5分）

In [33]:
AMZN_file = 'AMZN_2006-01-01_to_2018-01-01.csv'
IBM_file = 'IBM_2006-01-01_to_2018-01-01.csv'

AMZN_data = pd.read_csv(AMZN_file)
IBM_data = pd.read_csv(IBM_file)
# TODO
AMZN_s=pd.DataFrame(AMZN_data['Close'].tolist(),columns=['AMZN'],index=AMZN_data['Date'].to_list())
IBM_s=pd.DataFrame(IBM_data['Close'].to_list(),columns=['IBM'],index=IBM_data['Date'].to_list())
all_data=AMZN_s.join(IBM_s,how='outer')
all_data


,AMZN,IBM
2006-01-03,47.58,82.06
2006-01-04,47.25,81.95
2006-01-05,47.65,82.50
2006-01-06,47.87,84.95
2006-01-09,47.08,83.73
...,...,...
2017-12-22,1168.36,152.50
2017-12-26,1176.76,152.83
2017-12-27,1182.26,153.13
2017-12-28,1186.10,154.04
